In [2]:

%run 'Misc_Helper_Funcs.ipynb'

Helper Functions Included


In [1]:
print("Database_Interactions Included")

Database Interactions Included


In [ ]:
#update_scrape_table(mycursor)

In [10]:

class DatabaseManager:

    def __init__(self):
        #def default_newsdatabase_connect():
        self.mydb = mysql.connector.connect(
          host="localhost",
          user="pythonuser",
          passwd="password",
          database="newsdatabase"
        ) 

        self.mycursor = self.mydb.cursor(buffered=True)
#    return self.mycursor
#self.mycursor = default_newsdatabase_connect()

    def get_date_for_url_db(self, url):
        sql = "SELECT date FROM urls WHERE url= %s"
        val = (url,)

        self.mycursor.execute(self, sql,val)
        row = self.mycursor.fetchall()
        assert(1 == len(row))
        db_date = row[0][0]
        obj_date = convert_m_d_y_to_date_object(db_date)
        return obj_date

    def get_outlet_for_url_db(self, url):
        sql = "SELECT outlet FROM urls WHERE url= %s"
        val = (url,)

        self.mycursor.execute(sql,val)
        row = self.mycursor.fetchall()
        assert(1 == len(row))
        outlet = row[0][0]
        return outlet

    def set_path_for_file_db(self, url, file_loc):
        self.mycursor = self.mydb.cursor()
        sql = "UPDATE Rawtext SET File_Location = %s WHERE URL = %s"
        val = (file_loc, url)
        self.mycursor.execute(sql,val)
        self.mydb.commit()

    def mark_url_as_scraped_db(self, url):
        self.mycursor = self.mydb.cursor()
        sql = "UPDATE Rawtext SET Scraped = 1 WHERE URL = %s"
        val = (url,)
        self.mycursor.execute(sql,val)
        self.mydb.commit()

    def get_unscraped_urls_from_db(self):
        sql = "SELECT * FROM Rawtext WHERE Scraped = 0"

        self.mycursor.execute(sql)
        unscrapped = self.mycursor.fetchall()
        return unscrapped


    def check_present(self,url):
        t = (url,)
        #print(self.mycursor.execute('SELECT * FROM urls WHERE id=2'))
        self.mycursor.execute('SELECT * FROM urls WHERE url=%s', t)
        match = self.mycursor.fetchall()
        return len(match) > 0




    def update_unscraped_article(self, url, file_location):
        #self.mycursor = self.mydb.cursor()

        sql = "UPDATE Rawtext SET File_Location = %s WHERE URL = %s"
        val = (file_location, URL)

        self.mycursor.execute(sql, val)

        self.mydb.commit()

        print(self.mycursor.rowcount, "record(s) affected") 

    def check_article_rawtext_present(self, url):
        t = (url,)
        #print(self.mycursor.execute('SELECT * FROM urls WHERE id=2'))
        self.mycursor.execute('SELECT * FROM check WHERE url=%s', t)
        match = self.mycursor.fetchall()
        return len(match) > 0

    def write_raw_text_to_db(self, url, rawtext):
        if not check_article_rawtext_present(construct_full_url(url)):
            #print(construct_full_url('/2019/08/19/us/politics/elizabeth-warren-native-american.html', 'NewYorkTimes'))
            sql = "INSERT INTO raw_articles (url, raw_article) VALUES (%s, %s)"
            val = (construct_full_url(link, outlet),date,outlet) #("https://www.nytimes.com"+link, "08/20/2019",NewYorkTimes)
            self.mycursor.execute(sql, val)

            self.mydb.commit()

            print(self.mycursor.rowcount, "record inserted.")

    def update_scrape_table(self):
        #connect to db

        sql = "SELECT * FROM urls WHERE url NOT IN (SELECT URL FROM Rawtext)"

        self.mycursor.execute(sql)
        unscrapped = self.mycursor.fetchall()



        insert_sql = "INSERT INTO Rawtext (Scraped, URL, File_Location) VALUES (%s, %s, %s)"
        for x in unscrapped:

            URL=x[0]
            Scraped = 0
            File_Location= None

            insertval = (Scraped,URL,File_Location) #("https://www.nytimes.com"+link, "08/20/2019",NewYorkTimes)
            self.mycursor.execute(insert_sql, insertval)

            self.mydb.commit()
            print(URL+" Transered")

    def write_urls_to_db(self, url_list, outlet, category, date):
        for link in url_list:
            if not self.check_present(construct_full_url(link, outlet)):
                #print(construct_full_url('/2019/08/19/us/politics/elizabeth-warren-native-american.html', 'NewYorkTimes'))
                sql = "INSERT INTO urls (url, date, outlet) VALUES (%s, %s, %s)"
                val = (construct_full_url(link, outlet),date,outlet) #("https://www.nytimes.com"+link, "08/20/2019",NewYorkTimes)
                self.mycursor.execute(sql, val)

                self.mydb.commit()

                print(self.mycursor.rowcount, "record inserted.")

    #change to open new connection
    def open_connection_db(self):
        self.mydb = mysql.connector.connect(
          host="localhost",
          user="pythonuser",
          passwd="password",
          database="newsdatabase"
        ) 
        self.mycursor = self.mydb.cursor(buffered=True) 
        return self.mycursor